In [1]:
import pandas as pd

data = pd.read_csv("amazon_reviews_us_Mobile_Electronics_v1_00.tsv", sep="\t", error_bad_lines=False) 

C:\Users\Balddudehere\AppData\Local\Temp\ipykernel_23324\1054600650.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("amazon_reviews_us_Mobile_Electronics_v1_00.tsv", sep="\t", error_bad_lines=False)
Skipping line 35246: expected 15 fields, saw 22

Skipping line 87073: expected 15 fields, saw 22



In [2]:
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5.0,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5.0,0.0,1.0,N,Y,five star,it's great,2015-08-31
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5.0,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31
3,US,4332923,RRB9C05HDOD4O,B00QUFTPV4,221169481,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4.0,0.0,0.0,N,Y,Work very well but couldn't get used to not he...,Work very well but couldn't get used to not he...,2015-08-31
4,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2.0,0.0,0.0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104852 entries, 0 to 104851
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   marketplace        104852 non-null  object 
 1   customer_id        104852 non-null  int64  
 2   review_id          104852 non-null  object 
 3   product_id         104852 non-null  object 
 4   product_parent     104852 non-null  int64  
 5   product_title      104852 non-null  object 
 6   product_category   104852 non-null  object 
 7   star_rating        104850 non-null  float64
 8   helpful_votes      104850 non-null  float64
 9   total_votes        104850 non-null  float64
 10  vine               104850 non-null  object 
 11  verified_purchase  104850 non-null  object 
 12  review_headline    104848 non-null  object 
 13  review_body        104849 non-null  object 
 14  review_date        104850 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 1

In [4]:
text_column = 'review_body' 


In [5]:
from textblob import TextBlob

def analyze_sentiment(text):
    try:
        blob = TextBlob(text)
        polarity = blob.sentiment.polarity  # Polarity: -1 (negative) to 1 (positive)
        if polarity > 0:
            sentiment = 'Positive'
        elif polarity == 0:
            sentiment = 'Neutral'
        else:
            sentiment = 'Negative'
        return polarity, sentiment
    except Exception as e:
        return None, 'Error'

In [6]:
data['Polarity'], data['Sentiment'] = zip(*data[text_column].apply(analyze_sentiment))

In [7]:
output_file = 'sentiment_analysis_results.csv'
data.to_csv(output_file, index=False)

#### Data Ingestion and Database (sqlite3 for local ingestion since AWS is not working)

In [42]:
import sqlite3

conn = sqlite3.connect("sentiments.db")
data.to_sql("SentimentAnalysis", conn, if_exists="replace", index=False)

104852

In [43]:
from wordcloud import WordCloud

#### Wordcloud for Positive and Negative Sentiments

In [17]:
import io
import base64

positive_text = " ".join(data[data['Sentiment'] == 'Positive'][text_column].astype(str))
negative_text = " ".join(data[data['Sentiment'] == 'Negative'][text_column].astype(str))

In [18]:
def create_wordcloud(text, background_color='white'):
    wordcloud = WordCloud(width=800, height=400, background_color=background_color).generate(text)
    img = io.BytesIO()
    wordcloud.to_image().save(img, format='PNG')
    img.seek(0)
    return base64.b64encode(img.getvalue()).decode()

positive_wordcloud = create_wordcloud(positive_text, 'white')
negative_wordcloud = create_wordcloud(negative_text, 'black')

In [13]:
!python -m pip install dash

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/72/ef/d46131f4817f18b329e4fb7c53ba1d31774239d91266a74bccdc932708cc/dash-2.18.2-py3-none-any.whl.metadata
  Obtaining dependency information for dash-html-components==2.0.0 from https://files.pythonhosted.org/packages/75/65/1b16b853844ef59b2742a7de74a598f376ac0ab581f0dcc34db294e5c90e/dash_html_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-core-components==2.0.0 from https://files.pythonhosted.org/packages/00/9e/a29f726e84e531a36d56cff187e61d8c96d2cc253c5bcef9a7695acb7e6a/dash_core_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-table==5.0.0 from https://files.pythonhosted.org/packages/da/ce/43f77dc8e7bbad02a9f88d07bf794eaf68359df756a28bb9f2f78e255bb1/dash_table-5.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for retrying from

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Dashbaord using dash in python

In [14]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

C:\Users\Balddudehere\AppData\Local\Temp\ipykernel_23324\3649862469.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Balddudehere\AppData\Local\Temp\ipykernel_23324\3649862469.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [29]:
import plotly.express as px
import plotly.graph_objs as go

In [40]:
app = dash.Dash(__name__)
app.layout = html.Div([
    # Title Section
    html.Div([
        html.H1("Sentiment Analysis of Amazon Customer Reviews", style={
            'textAlign': 'center',
            'fontFamily': 'Arial, sans-serif',
            'color': '#2a4d6f',
            'margin-bottom': '30px'
        })
    ], style={'padding-top': '20px'}),  # Padding to give space above the title

    # Main content container
    html.Div([
        # Sentiment Distribution Bar Chart
        html.Div([
            dcc.Graph(
                id='sentiment-distribution',
                figure={
                    'data': [
                        go.Bar(
                            x=data['Sentiment'].value_counts().index,
                            y=data['Sentiment'].value_counts().values,
                            marker=dict(color=['#1f77b4', '#ff7f0e', '#2ca02c'])
                        )
                    ],
                    'layout': go.Layout(
                        title="Sentiment Distribution",
                        xaxis={'title': 'Sentiment'},
                        yaxis={'title': 'Count'},
                        template="plotly_white",
                        margin={'l': 40, 'r': 40, 't': 40, 'b': 40},
                    )
                }
            )
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),

        # Polarity Distribution Histogram
        html.Div([
            dcc.Graph(
                id='polarity-distribution',
                figure={
                    'data': [
                        go.Histogram(
                            x=data['Polarity'],
                            nbinsx=20,
                            marker=dict(color='skyblue'),
                            opacity=0.75
                        )
                    ],
                    'layout': go.Layout(
                        title="Polarity Distribution",
                        xaxis={'title': 'Polarity'},
                        yaxis={'title': 'Frequency'},
                        template="plotly_white",
                        margin={'l': 40, 'r': 40, 't': 40, 'b': 40},
                    )
                }
            )
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),

    ], style={'display': 'flex', 'justify-content': 'space-between', 'margin': '20px 0'}),  # Flex layout for charts

    # Wordcloud Section (Positive and Negative)
    html.Div([
        # Positive Sentiment Wordcloud
        html.Div([
            html.H3("Wordcloud - Positive Sentiments", style={'textAlign': 'center', 'color': '#007bff'}),
            html.Img(src=f"data:image/png;base64,{positive_wordcloud}", style={'width': '100%', 'height': 'auto'})
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px', 'border': '1px solid #ddd', 'border-radius': '8px'}),

        # Negative Sentiment Wordcloud
        html.Div([
            html.H3("Wordcloud - Negative Sentiments", style={'textAlign': 'center', 'color': '#dc3545'}),
            html.Img(src=f"data:image/png;base64,{negative_wordcloud}", style={'width': '100%', 'height': 'auto'})
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px', 'border': '1px solid #ddd', 'border-radius': '8px'}),

    ], style={'display': 'flex', 'justify-content': 'space-between', 'margin': '20px 0'}),  # Flexbox for wordclouds

])

In [41]:
if __name__ == '__main__':
    app.run_server(debug=True)